Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# トレーニング パイプライン - カスタム スクリプト
_**カスタム スクリプトを使用した Many models のトレーニング**_

----

このノートブックでは、カスタム スクリプトを使用して Many Models をトレーニングして登録するパイプラインを作成する方法を示します。[ParallelRunStep](https://docs.microsoft.com/ja-jp/azure/machine-learning/tutorial-pipeline-batch-scoring-classification) を利用して、モデルのトレーニング プロセスを並列化し、プロセスをより効率的にします。このソリューション アクセラレータでは、[OJ 販売データセット](https://azure.microsoft.com/ja-jp/services/open-datasets/catalog/sample-oj-sales-simulated/)を使用して、各店舗の売上とオレンジ ジュースのブランドを予測する個々のモデルをトレーニングします。

ここで使用するモデルは、scikit-learn と pandas ユーティリティに基づいて構築された単純な回帰ベースの予測です。予測がどのように構築されているかを確認するには、トレーニング スクリプトを参照してください。この予測はデモ用に意図されているので、時系列モデリングで遭遇するさまざまな特殊なケースは扱いません。たとえばここでのモデルではすべての時系列が、欠損値のない連続した間隔で定期的にサンプリングされた観測値で構成されていることを前提としています。このモデルでは、カテゴリ変数の処理は含まれません。欠損値、高度な特徴付け、および自動的なモデル選択を処理する、より一般的な予測機能については、[AutoML 予測タスク](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-auto-train-forecast)を参照してください。また、[Many Models シナリオにおける AutoML 予測](../Automated_ML)を示すノートブックを参照してください。

### 前提条件
この時点で、次の内容が完了している必要があります：

1. [00_Setup_AML_Workspace notebook](../../00_Setup_AML_Workspace.ipynb) を使用して、AML ワークスペースが作成されていること
2. [01_Data_Preparation.ipynb](../../01_Data_Preparation.ipynb) を実行してデータセットを作成していること

#### Azure ML SDK の最新バージョンを使用していることを確認し、パイプライン ステップ パッケージをインストールしてください。

In [ ]:
#!pip install --upgrade azureml-sdk

In [ ]:
# !pip install azureml-pipeline-steps

## 1.0 ワークスペースとデータストアへの接続

In [ ]:
from azureml.core import Workspace

# ワークスペースのセットアップ
ws = Workspace.from_config()

# データストアのセットアップ
dstore = ws.get_default_datastore()

print('Workspace Name: ' + ws.name, 
      'Azure Region: ' + ws.location, 
      'Subscription Id: ' + ws.subscription_id, 
      'Resource Group: ' + ws.resource_group, 
      sep = '\n')

## 2.0 実験の作成

In [ ]:
from azureml.core import Experiment

experiment = Experiment(ws, 'oj_training_pipeline')

print('Experiment name: ' + experiment.name)

## 3.0 トレーニング データセットの取得

次に、[Dataset.get_by_name()](https://docs.microsoft.com/python/api/azureml-core/azureml.core.dataset.dataset#get-by-name-workspace--name--version--latest--) メソッドを使用してトレーニング データセットを取得します。

これは[データ準備ノートブック](../01_Data_Preparation.ipynb)で作成および登録したトレーニング データセットです。ファイルのサブセットのみを使用するように選択した場合は、トレーニング データセット名を `oj_data_small_train` と指定します。それ以外の場合は、`oj_data_train` を指定します。

小さなデータセットから始め、すべてが正常に実行された後に、完全なデータセットにスケールアップすることをお勧めします。

In [ ]:
dataset_name = 'oj_data_small_train'
#dataset_name = 'oj_data_train'

In [ ]:
from azureml.core.dataset import Dataset

dataset = Dataset.get_by_name(ws, name=dataset_name)
dataset_input = dataset.as_named_input(dataset_name)

## 4.0 トレーニング パイプラインの作成
データセット、ワークスペースおよびデータストアが設定できたので、トレーニング用のパイプラインにまとめることができます。

### 4.1 ParallelRunStep の環境を構成する
[環境](https://docs.microsoft.com/ja-jp/azure/machine-learning/concept-environments)は、パイプラインを実行するために必要なリソースのコレクションを定義します。私たちは、[scikit-learn](https://scikit-learn.org/stable/index.html) Pythonライブラリを含むトレーニングスクリプト用の再現可能なPython環境を構成します。

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

train_env = Environment(name="many_models_environment")
train_conda_deps = CondaDependencies.create(pip_packages=['sklearn', 'pandas', 'joblib', 'azureml-defaults', 'azureml-core', 'azureml-dataprep[fuse]'])
train_env.python.conda_dependencies = train_conda_deps

### 4.2 コンピュート ターゲットの選択

現在 PipelineRunBuilder は AMLCompute のみをサポートしています。これは[セットアップ ノートブック](../00_Setup_AML_Workspace.ipynb#3.0-Create-compute-cluster)で作成したコンピュート クラスタです。

In [ ]:
cpu_cluster_name = "cpucluster"

In [ ]:
from azureml.core.compute import AmlCompute

compute = AmlCompute(ws, cpu_cluster_name)

### 4.3 ParallelRunConfig の設定

[ParallelRunConfig](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps.parallel_run_config.parallelrunconfig?view=azure-ml-py) は、次で作成する ParallelRunStep の構成を提供します。ここでは、上記で作成した環境とコンピュート ターゲットと、各バッチ用のエントリ スクリプトを指定します。

構成する重要なパラメーターには、以下の項目があります：
- **mini_batch_size**: バッチあたりのファイル数。500 個のファイルがあり、mini_batch_sizeが 10 の場合、それぞれ 10 個のファイルを含む 50 個のバッチが作成されます。バッチは、さまざまなノードに分割されます。

- **node_count**: ユーザー スクリプトの実行に使用するコンピューティング ノードの数。OJ データセットの小さなサンプルでは、1 つのノードのみが必要ですが、より多くのファイルで構成される大きなデータセットでは、この数を増やす必要があります。ここでノード数を 5 を超える場合は、コンピュート クラスターのmax_nodesも増やす必要があります。

- **process_count_per_node**: ノードあたりのプロセス数。使用予定のコンピュート クラスターには 8 つのコアがあるため、このパラメーターを 8 に設定します。

- **run_invocation_timeout**: run() メソッドの呼び出しタイムアウト (秒単位)。タイムアウトは、1 つのモデルの最大トレーニング時間 (秒単位) より大きく設定する必要があります。デフォルトは 60 秒です。トレーニングに最も時間がかかるバッチは約 120 秒であるため、メソッドの実行に十分な時間を確保するために 180 に設定します。


また、トレーニング クラスターのノード数、ノードあたりのプロセス数、データセット名に関する情報を保持するタグも追加します。'Tags' 列は、Azure Machine Learning Studio で確認できます。

In [ ]:
from azureml.pipeline.steps import ParallelRunConfig

processes_per_node = 8
node_count = 1
timeout = 180

parallel_run_config = ParallelRunConfig(
    source_directory='./scripts',
    entry_script='train.py',
    mini_batch_size="1",
    run_invocation_timeout=timeout,
    error_threshold=10,
    output_action="append_row",
    environment=train_env,
    process_count_per_node=processes_per_node,
    compute_target=compute,
    node_count=node_count)

### 4.4 ParallelRunStep の構成

この [ParallelRunStep](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps.parallel_run_step.parallelrunstep?view=azure-ml-py) はトレーニング パイプラインにおけるメインのステップです。

まず出力ディレクトリを設定し、パイプラインの出力名を定義します。パイプラインの出力データを格納するデータストアは、ワークスペースのデフォルトデータストアです。

In [ ]:
from azureml.pipeline.core import PipelineData

output_dir = PipelineData(name="training_output", datastore=dstore)

ここでは、ParallelRunConfig に対して名前と、上記で作成された他のいくつかのパラメータを与えます。

- **inputs**: 入力データセットのリスト。ここでは、前のノートブックで作成したデータセットを使用します。そのパス内のファイル数によって、ParallelRunStep でトレーニングされるモデルの数が決まります。

- **output**: 出力ディレクトリに対応する PipelineData オブジェクト。定義した出力ディレクトリを使用します。

- **arguments**: train.py 入力スクリプトに必要な引数のリスト。ここでは、時系列データのスキーマ(例：ターゲット、タイムスタンプ、id列名)、モデリング前に削除する必要がある列、モデルの種類を識別する文字列、そしてテスト用に残しておきたい観測値の数を与えます。

In [ ]:
from azureml.pipeline.steps import ParallelRunStep

parallel_run_step = ParallelRunStep(
    name="many-models-training",
    parallel_run_config=parallel_run_config,
    inputs=[dataset_input],
    output=output_dir,
    allow_reuse=False,
    arguments=['--target_column', 'Quantity', 
               '--timestamp_column', 'WeekStarting', 
               '--timeseries_id_columns', 'Store', 'Brand',
               '--drop_columns', 'Revenue', 'Store', 'Brand',
               '--model_type', 'lr',
               '--test_size', 20]
)

## 5.0 パイプラインの実行
次に、実行するパイプラインを送信します。この実行では、トレーニング セットを使用して各データセットのモデルをトレーニングし、テスト セットを使用して適合度の精度メトリックを計算し、最終的にすべてのデータを使用してモデルを再トレーニングします。10 ファイルの場合、これには数分しかかかりませんが、完全なデータセットでは 1 時間以上かかる場合があります。

In [ ]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace=ws, steps=[parallel_run_step])
run = experiment.submit(pipeline)

In [ ]:
# 実行が完了するまで待機
run.wait_for_completion(show_output=False, raise_on_error=True)

## 6.0 トレーニング パイプラインの結果を表示する
train.py の実行メソッドで返されるデータフレームは、*parallel_run_step.txt* に出力されます。トレーニング パイプラインの結果を確認するには、そのファイルをダウンロードし、DataFrame にデータを読み取り、サンプル内のメトリックを含む結果を視覚化します。Azure Machine Learning トレーニング用コンピュート クラスターに送信された実行には、しばらく時間がかかることがあります。出力は、実行が完了するまで生成されません。Azure ポータル https://ml.azure.com で実行の状態を監視できます。

### 6.1 ローカルに parallel_run_step.txt をダウンロードする

In [ ]:
import os

def download_results(run, target_dir=None, step_name='many-models-training', output_name='training_output'):
    stitch_run = run.find_step_run(step_name)[0]
    port_data = stitch_run.get_output_data(output_name)
    port_data.download(target_dir, show_progress=True)
    return os.path.join(target_dir, 'azureml', stitch_run.id, output_name)

file_path = download_results(run, 'output')
file_path

### 6.2 ファイルを dataframe に変換する

In [ ]:
import pandas as pd

df = pd.read_csv(file_path + '/parallel_run_step.txt', sep=" ", header=None)
df.columns = ['Store', 'Brand', 'Model', 'File Name', 'ModelName', 'StartTime', 'EndTime', 'Duration',
              'MSE', 'RMSE', 'MAE', 'MAPE', 'Index', 'Number of Models', 'Status']

df['StartTime'] = pd.to_datetime(df['StartTime'])
df['EndTime'] = pd.to_datetime(df['EndTime'])
df['Duration'] = df['EndTime'] - df['StartTime']
df.head()

### 6.3 結果を確認する

In [ ]:
total = df['EndTime'].max()  - df['StartTime'].min()

print('Number of Models: ' + str(len(df)))
print('Total Duration: ' + str(total)[6:])

In [ ]:
print('Average MAPE: ' + str(round(df['MAPE'].mean(), 5)))
print('Average MSE: ' + str(round(df['MSE'].mean(), 5)))
print('Average RMSE: ' + str(round(df['RMSE'].mean(), 5)))
print('Average MAE: '+ str(round(df['MAE'].mean(), 5)))

In [ ]:
print('Maximum Duration: '+ str(df['Duration'].max())[7:])
print('Minimum Duration: ' + str(df['Duration'].min())[7:])
print('Average Duration: ' + str(df['Duration'].mean())[7:])

### 6.4 モデル間でのパフォーマンスの視覚化

ここでは、テスト用にサブセットを使用して実行中に計算されたエラーメトリックからいくつかのチャートを生成します。

まず、すべてのモデルにおける平均絶対パーセンテージ誤差(MAPE)の分布を調べます：

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt

fig = sns.boxplot(y='MAPE', data=df)
fig.set_title('MAPE across all models')

次に、ブランドまたは店舗によってそれを分解して、モデル全体のエラーの変動を確認します

In [ ]:
fig = sns.boxplot(x='Brand', y='MAPE', data=df)
fig.set_title('MAPE by Brand')

また、異なるブランドのモデルがトレーニングにかかった時間を見ることもできます

In [ ]:
brand = df.groupby('Brand')
brand = brand['Duration'].sum()
brand = pd.DataFrame(brand)
brand['time_in_seconds'] = [time.total_seconds()  for time in brand['Duration']]

brand.drop(columns=['Duration']).plot(kind='bar')
plt.xlabel('Brand')
plt.ylabel('Seconds')
plt.title('Total Training Time by Brand')
plt.show()

## 7.0 パイプラインの発行とスケジュール (オプション)


### 7.1 パイプラインを発行する
満足できるパイプラインを作成したら、パイプラインを発行して、後からプログラムで呼び出すことができます。パイプラインの発行と呼び出しの詳細については、この[チュートリアル](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-create-machine-learning-pipelines#publish-a-pipeline)を参照してください。

In [ ]:
# published_pipeline = pipeline.publish(name = 'train_many_models',
#                                      description = 'train many models',
#                                      version = '1',
#                                      continue_on_step_failure = False)

### 7.2 パイプラインのスケジュール実行
また、時間ベースまたは変更ベースのスケジュールで実行するように[パイプラインをスケジュール](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-trigger-published-pipeline)することもできます。これは、毎月、またはデータドリフトなどの別のトリガーに基づいて、モデルを自動的に再トレーニングするために使用できます。

In [ ]:
# from azureml.pipeline.core import Schedule, ScheduleRecurrence
    
# training_pipeline_id = published_pipeline.id

# recurrence = ScheduleRecurrence(frequency="Month", interval=1, start_time="2020-01-01T09:00:00")
# recurring_schedule = Schedule.create(ws, name="training_pipeline_recurring_schedule", 
#                             description="Schedule Training Pipeline to run on the first day of every month",
#                             pipeline_id=training_pipeline_id, 
#                             experiment_name=experiment.name, 
#                             recurrence=recurrence)

## 次のステップ
モデルのトレーニングとスコア付けが終了したら、[03_CustomScript_Forecasting_Pipeline.ipynb](03_CustomScript_Forecasting_Pipeline.ipynb) に進み、作成したモデルを使用して予測を行います。